### **Exercícios**

**1 - Os projetistas de um banco de dados para um sistema de informação querem desenvolver uma fórmula para prever o tempo gasto em uma operação de busca nos dados dos últimos X dias. O tempo real gasto foi medido para diferentes valores de X, conforme mostrado abaixo. Apresente uma fórmula e a sua avaliação para essa previsão.**

Dias,Tempo

1,0.65

2,0.79

4,1.36

8,2.26

16,3.59

25,5.36

In [3]:
import numpy as np 
import matplotlib.pyplot as plt 

def estimate_coef(x, y): 
    # number of observations/points 
    n = np.size(x) 

    # mean of x and y vector 
    m_x, m_y = np.mean(x), np.mean(y) 

    # calculating cross-deviation and deviation about x 
    SS_xy = np.sum(y*x) - n*m_y*m_x 
    SS_xx = np.sum(x*x) - n*m_x*m_x 

    # calculating regression coefficients 
    b_1 = SS_xy / SS_xx 
    b_0 = m_y - b_1*m_x 

    return(b_0, b_1) 

def plot_regression_line(x, y, b): 
    # plotting the actual points as scatter plot 
    plt.scatter(x, y, color = "m",marker = "o", s = 30) 

    # predicted response vector 
    y_pred = b[0] + b[1]*x 

    # plotting the regression line 
    plt.plot(x, y_pred, color = "g") 

    # putting labels 
    plt.xlabel('x') 
    plt.ylabel('y') 

    # function to show plot 
    plt.show() 

def main(): 
    # observations 
    x = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]) 
    y = np.array([1, 3, 2, 5, 7, 8, 8, 9, 10, 12]) 

    # estimating coefficients 
    b = estimate_coef(x, y) 
    print("Estimated coefficients:\nb_0 = {} b_1 = {}".format(b[0], b[1])) 

    # plotting regression line 
    plot_regression_line(x, y, b) 

if __name__ == "__main__": 
    main() 


Estimated coefficients:
b_0 = 1.2363636363636363 b_1 = 1.1696969696969697


<Figure size 640x480 with 1 Axes>

**2- Descreva as métricas de avaliação de desempenho de modelos: Recall e F1-score.  
Qual a vantagem sobre a acurácia e precisão?**

**3-  Utilize o dataset "weather_nominal.csv" disponível no moodle. Considere 70% das instâncias para treino e 30% para teste.  
Construa um modelo utilizando árvore de decisão para prever se uma pessoa vai jogar golf ou não.  
Qual o desempenho?  
Tente melhorar esse modelo e descreva como.**